In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as Fun
from scipy import signal
from scipy import signal

In [4]:
import numpy as np
import pandas as pd
# import ast
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
import os

from sklearn.utils import shuffle
import math
# import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,normalize, MinMaxScaler


import os
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score
import warnings

In [5]:

ttt = np.load('y_report_all.npy', allow_pickle = True)
print(len(ttt))

21396


In [28]:
data_filepath = "./ECG_data/"

X_test = np.load('X_all.npy')
y_test = np.load('y_all.npy', allow_pickle=True)
y_report = np.load('y_report_all.npy', allow_pickle = True)

print('X_test:', X_test.shape)
print('y_test:', y_test.shape)
print('y_report:', y_report.shape)

X_test: (21396, 1000, 12)
y_test: (21396,)
y_report: (21396,)


In [30]:
# reshape y_train, y_test
y_test = y_test.reshape(len(y_test), 1)
y_report = y_report.reshape(len(y_report), 1)

In [14]:
y_test

array([list(['NORM']), list(['NORM']), list(['NORM']), ...,
       list(['STTC']), list(['NORM']), list(['NORM'])], dtype=object)

In [31]:
num2class = np.array(['NORM', 'MI', 'STTC', 'CD', 'HYP'])
    
for index in range(len(y_test)):
    y_test[index] = np.where(num2class == y_test[index][0][0])[0][0]

y_test = y_test.astype('uint8')

print(y_test)
print('y_test:', y_test.shape)

[[0]
 [0]
 [0]
 ...
 [2]
 [0]
 [0]]
y_test: (21396, 1)


In [32]:
np.save('y_test.npy', y_test)

In [ ]:
train_all=np.concatenate((y_test,y_report,X_test),axis=1)
df = pd.DataFrame(train_all, index = None)
df

In [6]:
# def apply_scaler(X, scaler):
#     X_tmp = []
#     for x in X:
#         x_shape = x.shape
#         X_tmp.append(scaler.transform(x.flatten()[:,np.newaxis]).reshape(x_shape))
#     X_tmp = np.array(X_tmp)
#     return X_tmp


# scaler = StandardScaler()

# scaler.fit(np.vstack(X_test).flatten()[:,np.newaxis].astype(float))

'''
X_train_scale = apply_scaler(X_train, scaler)
X_test_scale = apply_scaler(X_test, scaler)
X_val_scale = apply_scaler(X_val, scaler)

del X_train, X_test, X_val

'''

#X_train_scale = apply_scaler(X_train, scaler)
# X_test_scale = apply_scaler(X_test, scaler)

'\nX_train_scale = apply_scaler(X_train, scaler)\nX_test_scale = apply_scaler(X_test, scaler)\nX_val_scale = apply_scaler(X_val, scaler)\n\ndel X_train, X_test, X_val\n\n'

In [7]:
#del X_train, X_test

In [8]:
#def shuffle_train_test(X_train, y_train, X_test, y_test):
#    shuffled_indices_x = np.random.permutation(len(y_train))
#    shuffled_indices_y = np.random.permutation(len(y_test))

#    return X_train[shuffled_indices_x], y_train[shuffled_indices_x], X_test[shuffled_indices_y], y_test[shuffled_indices_y]

#X_train, y_train, X_test, y_test = shuffle_train_test(X_train, y_train, X_test, y_test)

In [10]:
##count the number of each class

In [34]:
np.set_printoptions(threshold=20)
print(y_test[1:20])

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [2]
 [0]]


In [29]:
y_report

array([['sinusrhythmus periphere niederspannung'],
       ['sinusbradykardie sonst normales ekg'],
       ['sinusrhythmus normales ekg'],
       ...,
       ['sinusrhythmus lagetyp normal t abnorm in anterioren ableitungen 4.46                          unbestÄtigter bericht Edit: INJAS 50, (ISCAS)'],
       ['supraventrikulÄre extrasystole(n) sinusrhythmus linkstyp t abnorm in hochlateralen ableitungen 4.46                          unbestÄtigter bericht'],
       ['sinusrhythmus p-sinistrocardiale lagetyp normal 4.46                          unbestÄtigter bericht']],
      dtype=object)

In [7]:
sampling_rate = 100

def checkR(ecg):
    max_val = np.max(ecg)
    min_val = np.min(ecg)
    threshold_val = (max_val-min_val)*0.7 + min_val
    index = []
    for i in range(1, len(ecg)-2):
        if ecg[i] == np.max(ecg[i-1:i+2]) and ecg[i] > threshold_val:
            if index != []:
                if i-index[-1] <= 60.0/60.0*sampling_rate and i-index[-1] >= 60.0/160.0*sampling_rate:
                    index.append(i)
            else:
                index.append(i)
    return np.array(index)

In [8]:

def splitByR(ecg): #
    index = checkR(ecg)
    ecg_tmp=None
    #ecg_rhythm = None
    ecg_rhythm = []
    for i in range(len(index)):
        #if index[i]>200 and index[i]<1000-350:
        if index[i]>35 and index[i]<1000-65:
            #ecg_rhythm = ecg[index[i]-150:index[i]+350]
            ecg_tmp = ecg[index[i]-35:index[i]+65]
            ecg_rhythm.append(ecg_tmp)
            continue
    ecg_rhythm=np.array(ecg_rhythm)
    return ecg_rhythm

In [9]:
len(X_test)

21396

In [10]:
X_test_pieces=[]
y_test_pieces=[]
y_report_pieces = []

for i in range (0, len(X_test)):
    label_tmp=y_test[i]
    report_tmp = y_report[i]
    ecg_tmp=None
    tmp=X_test[i] 
    temp=X_test[i][:,1] 
    temp_2=temp.tolist()
    index = checkR(temp_2)
    for j in range (0, len(index)):
        index=np.array(index)
        tmp_index= index[j]
        if tmp_index>35 and tmp_index<1000-65:
            all_lead_tmp=[]
            for k in range (0,12):
                current_lead = tmp[:,k]
                ecg_tmp = current_lead[tmp_index-35:tmp_index+65]
                all_lead_tmp.append(ecg_tmp)
                    
            X_test_pieces.append(all_lead_tmp)
            y_test_pieces.append(label_tmp)
            y_report_pieces.append(report_tmp)


In [11]:
X_test_pieces=np.array(X_test_pieces)
y_test_pieces=np.array(y_test_pieces)
y_report_pieces = np.array(y_report_pieces)

In [12]:
X_test_pieces.shape

(170086, 12, 100)

In [13]:
y_test_pieces.shape

(170086, 1)

In [14]:
np.zeros([12, 100])[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
np.unique(y_test_pieces)

array([0, 1, 2, 3, 4], dtype=uint8)

In [16]:
false_count = 0
for index in range(len(y_test_pieces)):
    if y_test_pieces[index].any() == np.zeros([12, 100]).any():
        false_count += 1
false_count

66775

In [17]:
X_test_pieces=np.reshape(X_test_pieces,(len(X_test_pieces),1200))

In [18]:
X_test_pieces.shape

(170086, 1200)

In [19]:
X_test_pieces_600=[]

for i in range(0, len(X_test_pieces)):
    tmp=X_test_pieces[i]
    tmp_resampled = signal.resample(tmp, 600)
    X_test_pieces_600.append(tmp_resampled)
X_test_pieces_600=np.array(X_test_pieces_600)

In [20]:
X_test_pieces_600.shape

(170086, 600)

In [21]:
X_test_norm=[]

    
for j in range(0,len(X_test_pieces_600)):
    tmp=X_test_pieces_600[j]
    tmp2=tmp
    tmp3 = [float('{:.6f}'.format(i)) for i in tmp2]
    tmp4=np.array(tmp3)
    tmp5=tmp4/np.linalg.norm(tmp4)
    X_test_norm.append(tmp5.T)

In [22]:
X_test_norm=np.array(X_test_norm)
print("norm-ed!")

norm-ed!


In [23]:
train_all=np.concatenate((y_test_pieces,y_report_pieces,X_test_norm),axis=1)

# np.savetxt('data_600.csv',train_all, fmt='%.6f', delimiter=",")

In [24]:
df = pd.DataFrame(train_all, index = None)
df

,0,1,2,3,4,5,6,7,8,9,...,592,593,594,595,596,597,598,599,600,601
0,0,sinusbradykardie sonst normales ekg,-0.015594,-0.008677,-0.010193,-0.008157,-0.009297,-0.009295,-0.010226,-0.007686,...,-0.024029,-0.024676,-0.025722,-0.025821,-0.026652,-0.027922,-0.028798,-0.0299,-0.029236,-0.031036
1,0,sinusrhythmus normales ekg,-0.032629,-0.034772,-0.033286,-0.037921,-0.041388,-0.042681,-0.046535,-0.053703,...,-0.011798,-0.007505,-0.007396,-0.00827,-0.012079,-0.01581,-0.019725,-0.021014,-0.02207,-0.022972
2,0,sinusrhythmus normales ekg,-0.035144,-0.033342,-0.042117,-0.041946,-0.039102,-0.037911,-0.03987,-0.036804,...,-0.050231,-0.04876,-0.048505,-0.045222,-0.043496,-0.040763,-0.04053,-0.040473,-0.041805,-0.042707
3,0,sinusrhythmus normales ekg,-0.025497,-0.037556,-0.034812,-0.034122,-0.03055,-0.030554,-0.027667,-0.027289,...,0.032029,0.021629,0.006849,0.001163,-0.001073,-0.000422,-0.002786,-0.000978,-0.003434,0.000335
4,0,sinusrhythmus normales ekg,-0.016326,-0.014744,-0.018593,-0.018802,-0.01587,-0.015734,-0.017245,-0.016718,...,-0.014507,-0.016445,-0.016798,-0.017465,-0.017582,-0.017896,-0.017275,-0.018279,-0.019072,-0.020776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170081,0,supraventrikulÄre extrasystole(n) sinusrhythmu...,0.004663,-0.026157,-0.012642,-0.021593,0.041584,0.098006,0.08371,0.00608,...,0.011945,0.009991,0.002308,-0.007897,-0.019231,-0.021934,-0.028703,-0.004954,0.083051,0.087554
170082,0,supraventrikulÄre extrasystole(n) sinusrhythmu...,0.013785,-0.016634,-0.010602,0.01343,0.095463,0.117048,0.051543,-0.016926,...,-0.009342,0.004341,0.008867,0.003509,-0.013591,-0.0192,-0.022962,-0.02103,-0.004673,0.095656
170083,0,supraventrikulÄre extrasystole(n) sinusrhythmu...,-0.026209,-0.012936,-0.020346,0.011272,0.089377,0.110931,0.044668,-0.015617,...,-0.026963,-0.031235,-0.04327,-0.050484,-0.053697,-0.051163,-0.054655,0.019179,0.069906,0.015241
170084,0,supraventrikulÄre extrasystole(n) sinusrhythmu...,-0.005359,-0.004222,-0.007079,-0.008478,-0.014456,-0.01083,-0.01254,-0.009341,...,-0.0021,0.006199,0.004301,-0.000758,-0.008075,-0.010305,-0.013704,-0.012378,-0.012152,-0.007051


In [26]:
df1 = df.loc[0]

df1

0                                        0
1      sinusbradykardie sonst normales ekg
2                                -0.015594
3                                -0.008677
4                                -0.010193
                      ...                 
597                              -0.027922
598                              -0.028798
599                                -0.0299
600                              -0.029236
601                              -0.031036
Name: 0, Length: 602, dtype: object

In [27]:

df1.to_csv('df1.csv', index = False)

In [29]:
df.to_csv('data_600.csv', index = False)

In [30]:
len(y_report_pieces)

170086

In [31]:
#extract frequency domain features

In [32]:
X_test_raw=np.reshape(X_test_pieces,(len(X_test_pieces),12,100))

In [33]:
len(X_test_raw)

170086

In [34]:
from time_frequency_domain_features import *



def func_timefeatures(sequence_data):
    time_features=[]
    
    result_mean_fea=mean_fea(sequence_data)
    time_features.append(result_mean_fea)
    
    result_rms_fea=rms_fea(sequence_data)
    time_features.append(result_rms_fea)
    
    result_sr_fea=sr_fea(sequence_data)
    time_features.append(result_sr_fea)
    
    result_am_fea=am_fea(sequence_data)
    time_features.append(result_am_fea)
    
    result_skew_fea=skew_fea(sequence_data)
    time_features.append(result_skew_fea)
    
    result_kurt_fea=kurt_fea(sequence_data)
    time_features.append(result_kurt_fea)
    
    result_max_fea=max_fea(sequence_data)
    time_features.append(result_max_fea)
    
    result_min_fea=min_fea(sequence_data)
    time_features.append(result_min_fea)
    
    result_pp_fea=pp_fea(sequence_data)
    time_features.append(result_pp_fea)
    
    result_var_fea=var_fea(sequence_data)
    time_features.append(result_var_fea)
    
    result_waveform_index_fea=waveform_index(sequence_data)
    time_features.append(result_waveform_index_fea)
    
    result_peak_index_fea=peak_index(sequence_data)
    time_features.append(result_peak_index_fea)
    
    result_impluse_factor_fea=impluse_factor(sequence_data)
    time_features.append(result_impluse_factor_fea)
    
    #result_tolerance_index_fea=tolerance_index(sequence_data)
    #features.append(result_tolerance_index_fea)
    
    #result_skew_index_fea=skew_index(sequence_data)
    #features.append(result_skew_index_fea)
                    
    #result_kurt_index_fea=kurt_index(sequence_data)
    #features.append(result_kurt_index_fea)

    time_features=np.array(time_features)
    return time_features

In [35]:
from time_frequency_domain_features import fft_mean,fft_var,fft_std,fft_entropy,fft_energy,fft_skew,fft_kurt,fft_shape_mean,fft_shape_std,fft_shape_skew,fft_shape_kurt


def func_frequencyfeatures(sequence_data):
    fre_features=[]
    
    result_fft_mean=fft_mean(sequence_data)
    fre_features.append(result_fft_mean)
    
    result_fft_var=fft_var(sequence_data)
    fre_features.append(result_fft_var)
    
    result_fft_std=fft_std(sequence_data)
    fre_features.append(result_fft_std)
    
    result_fft_entropy=fft_entropy(sequence_data)
    fre_features.append(result_fft_entropy)
    
    result_fft_energy=fft_energy(sequence_data)
    fre_features.append(result_fft_energy)
    
    result_fft_skew=fft_skew(sequence_data)
    fre_features.append(result_fft_skew)
    
    result_fft_kurt=fft_kurt(sequence_data)
    fre_features.append(result_fft_kurt)
    
    result_fft_shape_mean=fft_shape_mean(sequence_data)
    fre_features.append(result_fft_shape_mean)
    
    result_fft_shape_std=fft_shape_std(sequence_data)
    fre_features.append(result_fft_shape_std)
    
    #result_fft_shape_skew=fft_shape_skew(sequence_data)
    #fre_features.append(result_fft_shape_skew)
    
    #result_fft_shape_kurt=fft_shape_kurt(sequence_data)
    #fre_features.append(result_fft_shape_kurt)

    fre_features=np.array(fre_features)
    return fre_features

In [36]:
len(X_test_raw[0])

12

In [37]:
X_test_time_features=[]
X_test_time_features=np.array(X_test_time_features)
X_test_fre_features=[]
X_test_fre_features=np.array(X_test_fre_features)


for i in tqdm(range(0, len(X_test_raw))):
    
    tmp_raw=X_test_raw[i]
    time_features_tmp=[]
    fre_features_tmp=[]
    for j in range(0,12):
        tmp_lead=tmp_raw[j]
        result_time_features=func_timefeatures(tmp_lead)
        result_fre_features=func_frequencyfeatures(tmp_lead)
        time_features_tmp.append(result_time_features)
        fre_features_tmp.append(result_fre_features)
    time_features_tmp=np.array(time_features_tmp)
    fre_features_tmp=np.array(fre_features_tmp)
    X_test_time_features=np.append(X_test_time_features,time_features_tmp)
    X_test_fre_features=np.append(X_test_fre_features,fre_features_tmp)

 57%|█████████████████████████████▊                      | 97462/170086 [46:07<50:24, 24.01it/s]/home/ubuntu/code/Transfer_Knowledge_from_Language_to_Electrocardiography/time_frequency_domain_features.py:120: RuntimeWarning: invalid value encountered in divide
  pr_freq = freq_spectrum * 1.0 / _freq_sum_
/home/ubuntu/code/Transfer_Knowledge_from_Language_to_Electrocardiography/time_frequency_domain_features.py:179: RuntimeWarning: invalid value encountered in double_scalars
  var = np.sum([0 if _freq_sum_ < epsn else np.power((x - shape_mean), 2) * freq_spectrum[x]
100%|█████████████████████████████████████████████████| 170086/170086 [1:48:56<00:00, 26.02it/s]


In [38]:
len(X_test_time_features)

26533416

In [39]:
X_test_time_features=np.array(X_test_time_features)
X_test_fre_features=np.array(X_test_fre_features)
X_test_time_features=np.reshape(X_test_time_features,(len(X_test_raw),156))
X_test_fre_features=np.reshape(X_test_fre_features,(len(X_test_raw),108))

In [40]:
X_test_time_features.shape

(170086, 156)

In [41]:
X_test_fre_features.shape

(170086, 108)

In [42]:
test_all=np.concatenate((y_test_pieces,X_test_norm,X_test_time_features,X_test_fre_features),axis=1)

np.savetxt('test.csv',test_all, fmt='%.6f', delimiter=",")

In [2]:
import os

if not os.path.exists("pngs"):
    os.makedirs("pngs")
